In [ ]:
exdir_path = r"/home/mikkel/expipe_temp_storage/mikkel_septum_entorhinal/1775-160817-01/main.exdir"
channel_group = 0

In [ ]:
import expipe
import os
from expipe_plugin_cinpla.config import load_parameters
import quantities as pq
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import neo
import exdir
import exana.tracking as tr

%matplotlib inline
rc = {'figure.figsize': (16, 9),
        'xtick.labelsize': 20,
        'ytick.labelsize': 20,
        'axes.labelsize': 30, 
        'axes.titlesize': 25,
        'font.size': 25,
        'legend.frameon': False,
        'legend.fontsize': 15,
        'image.cmap': 'jet'}
sns.set(rc=rc)
sns.set_color_codes()
plt.rcParams.update(rc)

PAR = load_parameters()
par = PAR.ANALYSIS_PARAMS

In [ ]:
# Load tracking
x, y, t, ang, ang_t = tr.get_processed_tracking(exdir_path, par)

In [ ]:
# Load electrophysiology
io = neo.ExdirIO(exdir_path)
blk = io.read_block()
seg = blk.segments[0]
chxs = blk.channel_indexes

In [ ]:
chx = chxs[channel_group]

# Spikes

In [ ]:
from expipe_plugin_cinpla import make_spatiality_overview
from exana.waveform import (plot_amp_clusters, plot_waveforms)
from exana.statistics import correlogram, plot_isi_hist
from exana.stimulus import plot_psth, epoch_overview

if len(blk.segments[0].epochs)==1:
    epo = blk.segments[0].epochs[0]
    epo_over = epoch_overview(epo, np.median(np.diff(epo.times)))
    stim_micro = True
    stim_macro = True
else:
    epo = None
    epo_over = None
    stim_micro = False
    stim_macro = False
    
color = 'b'
waveform = True
corr = True
spatial = True

In [ ]:
for unit in chx.units:
    if unit.annotations['cluster_group'].lower() == 'noise':
        continue
    sptr = unit.spiketrains[0]
    if spatial:
        make_spatiality_overview(x, y, t, ang, ang_t, sptr, acorr=None,
                                 G=None, fig=None, mask_unvisited=False,
                                 vmin=0, ang_binsize=2, projection='polar',
                                 title=unit.name, 
                                 rate_map=None, params=par)
    if waveform:
        fig = plt.figure()
        plot_waveforms(sptr=sptr, color=color, fig=fig)
    if corr:
        fig, ax = plt.subplots()
        bin_width = par['corr_bin_width'].rescale('s').magnitude
        limit = par['corr_limit'].rescale('s').magnitude
        count, bins = correlogram(t1=sptr.times.magnitude, t2=None,
                                  bin_width=bin_width, limit=limit,
                                  auto=True)
        ax.bar(bins[:-1] + bin_width / 2., count, width=bin_width,
                color=color)
        ax.set_xlim([-limit, limit])
    if stim_micro:
        fig = plt.figure()
        t_start = -np.round(epo.durations[0].rescale('ms')) * 3  # FIXME is milliseconds always good?
        t_stop = np.round(epo.durations[0].rescale('ms')) * 3
        binsize = (abs(t_start) + abs(t_stop)) / 100.
        plot_psth(sptr=sptr, epo=epo, t_start=t_start,
                  t_stop=t_stop, output='counts', binsize=binsize,
                  fig=fig)
    if stim_macro:
        t_start = -np.round(epo_over.durations[0])
        t_stop = np.round(epo_over.durations[0])
        binsize = (abs(t_start) + abs(t_stop)) / 100.
        fig = plt.figure()

        plot_psth(sptr=sptr, epo=epo_over, t_start=t_start,
                  t_stop=t_stop, output='counts', binsize=binsize,
                  fig=fig, ylim=None)

# LFP

In [ ]:
from exana.time_frequency import plot_psd, plot_tfr

psd = True
tfr = True

anas = chx.analogsignals

In [ ]:
xlim = [0, 100]

if psd:
    plot_psd(anas, color=color, xlim=xlim, nperseg=2048, legend=True)

In [ ]:
# DOWNSAMPLING
import scipy.signal as ss
import neo

ana = anas[-1]
if ana.sampling_rate > 250 *pq.Hz:
    q = int(ana.sampling_rate / 250 * pq.Hz)
    signal = ss.decimate(ana.magnitude.T, q=q, zero_phase=True)
    sampling_rate = ana.sampling_rate / q
    t_stop = len(signal) / sampling_rate
    ana = neo.AnalogSignal(signal.T * ana.units, t_start=0 * pq.s, sampling_rate=sampling_rate)

In [ ]:
if tfr:
    plot_tfr(ana, epo=epo_over, f0=3, flim=[0, 100], tlim=[80, 200] * pq.s, plot_ana=True)